# 3. Language Models - Neural Language Models

In [17]:
import random
random.seed(42)
%load_ext autoreload
%autoreload 2

In [22]:
# import torchtext
# from torchtext import data
# import spacy
# spacy_en = spacy.load('en')

# def spacy_tokenizer(text): # create a tokenizer function
#     return [tok.text for tok in spacy_en.tokenizer(text)]

# from torchtext.data.utils import get_tokenizer

# from dataloaders.lm_dataloader import load_quotes_db

# train_ds, valid_ds, test_ds = load_quotes_db(
#                                     train_path="data/quotesdb/funny_quotes.train.txt",
#                                     valid_path="data/quotesdb/funny_quotes.val.txt",
#                                     test_path="data/quotesdb/funny_quotes.test.txt",
#                                     encoding="utf-8",
#                                     tokenizer=spacy_tokenizer,
#                                     vocab=None)

In [32]:
# dataloader = DataLoader(, batch_size=args.batch_size * args.bptt,
#                             shuffle=False, collate_fn=lambda b: collate_batch(b, args, mask_id, cls_id))

# train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
#     (train_ds, valid_ds, test_ds),
#     batch_size=32,
#     bptt_len=30, # this is where we specify the sequence length
#     device=0,
#     repeat=False)

In [18]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
class LM(nn.Module):
  
  def __init__(self, hid_size, vocab_size, n_head, n_layers, pf_size, max_len, device):
    super().__init__()

    self.device = device
    
    self.hid_size = hid_size
    self.pf_size = pf_size
    self.max_len = max_len

    self.embedding = nn.Embedding(vocab_size, hid_size)

    self.position_enc = nn.Embedding(self.max_len, self.hid_size)
    self.position_enc.weight.data = self.position_encoding_init(self.max_len, self.hid_size)
    self.scale = torch.sqrt(torch.FloatTensor([self.hid_size])).to(device)

    self.layer_norm = nn.LayerNorm(self.hid_size)
    self.decoder_layer = nn.TransformerDecoderLayer(d_model=hid_size, nhead = n_head, dim_feedforward=self.pf_size)
    self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=n_layers, norm=self.layer_norm)
    self.fc = nn.Linear(hid_size, vocab_size)

    self._init_weights()
  
  def forward(self, x):
    sent_len, batch_size = x.shape[0], x.shape[1]
    memory_mask = self.generate_complete_mask(sent_len)
    tgt_mask = self.generate_triangular_mask(sent_len)
    memory = torch.zeros(1, batch_size, self.hid_size, device=self.device)

    temp = x
    temp = self.embedding(temp)

    pos = torch.arange(0,sent_len).unsqueeze(1).repeat(1,batch_size).to(self.device)
    temp_pos_emb = self.position_enc(pos)

    temp = temp * self.scale + temp_pos_emb
    temp = self.decoder(temp, memory, tgt_mask=tgt_mask)
    temp = self.fc(temp)
    return temp

  def _init_weights(self):
    for p in self.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)

  def append_decoder_layer(self):
    appended_mod = nn.TransformerDecoderLayer(d_model=hid_size, nhead = n_head).to(self.device)
    for p in appended_mod.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    model.decoder.layers.append(appended_mod)
    model.decoder.num_layers += 1

  def generate_triangular_mask(self, size):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).to(device)
        return
        
  def generate_complete_mask(self, size):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = torch.empty(size, size).to(device)
        mask.fill_(float('-inf'))
        return mask

  def generate_sequence(self, src):
    #src = [sent_len]
    src = src.unsqueeze(1)
    #src = [sent_len, 1]
    generate_step = 0
    while generate_step < 20:
      out = self.forward(src)
      #out = [sent_len + 1, 1, vocab_size]
      out = torch.argmax(out[-1, :], dim=1) # [1]
      out = out.unsqueeze(0) #[1,1]
      src = torch.cat((src, out), dim=0)
      generate_step += 1
    src = src.squeeze(1)
    return src
  
  def position_encoding_init(self, n_position, d_pos_vec):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2*i/d_pos_vec) for i in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    temp = torch.from_numpy(position_enc).type(torch.FloatTensor)
    temp = temp.to(self.device)
    return temp

In [3]:
import torchtext
from torchtext import data
import spacy
 
my_tok = spacy.load('en')
 
def spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]
 
TEXT = data.Field(lower=True, tokenize=spacy_tok)

C:\Users\manujoseph\Anaconda3\envs\bot\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [4]:
from torchtext.datasets import LanguageModelingDataset
train_dataset = LanguageModelingDataset("data/quotesdb/funny_quotes.train.txt", TEXT)
val_dataset = LanguageModelingDataset("data/quotesdb/funny_quotes.val.txt", TEXT)
test_dataset = LanguageModelingDataset("data/quotesdb/funny_quotes.test.txt", TEXT)

C:\Users\manujoseph\Anaconda3\envs\bot\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


In [5]:
TEXT.build_vocab(train_dataset)
print(len(TEXT.vocab))

46083


In [6]:
def make_train_iter(batch_size, bptt_len):
  train_iter = data.BPTTIterator(
    train_dataset,
    batch_size=batch_size,
    bptt_len=bptt_len, # this is where we specify the sequence length
    device=device,
    repeat=False,
    shuffle=True)
  print(len(train_iter))
  return train_iter

In [7]:
vocab_size = len(TEXT.vocab)
hid_size = 16
pf_size = 64
max_len = 512
n_head = 4
n_layer= 1
model = LM(hid_size, vocab_size, n_head, n_layer, pf_size, max_len, device).to(device)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,537,763 trainable parameters


In [9]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [10]:
def train_one_epoch(model,train_iter, optimizer, criterion, clip):
  epoch_loss = 0
  model.train()
  for batch in train_iter:
    optimizer.zero_grad()
    batch_text = batch.text
    batch_target = batch.target
    result = model(batch_text)
    loss = criterion(result.view(-1, result.shape[-1]), batch_target.view(-1))
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
    return epoch_loss / len(train_iter)
    print("epoch is {} loss is {}".format(epoch, epoch_loss / len(train_iter)))

In [11]:
def train(model, train_iter, optimizer, criterion, clip, N_EPOCH):
  for epoch in range(N_EPOCH):
    epoch_loss = train_one_epoch(model, train_iter, optimizer, criterion, clip)
    print("epoch is {} loss is {}".format(epoch, epoch_loss))

In [12]:
torch.cuda.empty_cache()

In [13]:
# for i in range(1, 6):
optimizer = NoamOpt(hid_size, 1, 2000,
          torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.98), eps=1e-9)
criterion = torch.nn.CrossEntropyLoss()
train_iter = make_train_iter(64, 6)
train(model, train_iter, optimizer, criterion, 1, 500)

3199


C:\Users\manujoseph\Anaconda3\envs\bot\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BPTTIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
C:\Users\manujoseph\Anaconda3\envs\bot\lib\site-packages\torchtext\data\batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


epoch is 0 loss is 0.003361069362362536
epoch is 1 loss is 0.003311016217512278
epoch is 2 loss is 0.0032280897788607953
epoch is 3 loss is 0.003154097292042405
epoch is 4 loss is 0.0030716204128998746
epoch is 5 loss is 0.002980860668109633
epoch is 6 loss is 0.0028838386905308848
epoch is 7 loss is 0.002786394580150627
epoch is 8 loss is 0.002680261867722335
epoch is 9 loss is 0.002570513897890149
epoch is 10 loss is 0.0024583458192425243
epoch is 11 loss is 0.002345041171876145
epoch is 12 loss is 0.0022309323555605603
epoch is 13 loss is 0.0021198197579749044
epoch is 14 loss is 0.0020159621207406276
epoch is 15 loss is 0.0019205824663282075
epoch is 16 loss is 0.0018345511455243734
epoch is 17 loss is 0.0017609125228552716
epoch is 18 loss is 0.0016973400980504314
epoch is 19 loss is 0.0016421805176372712
epoch is 20 loss is 0.0015921920640426116
epoch is 21 loss is 0.0015471985504827712
epoch is 22 loss is 0.0015043487918492443
epoch is 23 loss is 0.0014646929031090648
epoch is 2

In [16]:
source_sentence = ["when","life", "hands", "you", "lemons"]
print(source_sentence)
model.eval()
print(' '.join(source_sentence))
print()
x = TEXT.numericalize([source_sentence]).to(device).squeeze(1)
generated_sequence =model.generate_sequence(x)
words = [TEXT.vocab.itos[word_idx] for word_idx in generated_sequence]
print(' '.join(words))

['when', 'life', 'hands', 'you', 'lemons']
when life hands you lemons

when life hands you lemons irrational , leaving but the best , you love may be blind , love may be blind , love may


In [19]:
source_sentence = ["life","is", "a"]
print(source_sentence)
model.eval()
print(' '.join(source_sentence))
print()
x = TEXT.numericalize([source_sentence]).to(device).squeeze(1)
generated_sequence =model.generate_sequence(x)
words = [TEXT.vocab.itos[word_idx] for word_idx in generated_sequence]
print(' '.join(words))

['life', 'is', 'a']
life is a

life is a series of moments that old monkey english , anything , anything , anything , anything , anything , anything ,


In [20]:
source_sentence = ["i","really", "need","a"]
print(source_sentence)
model.eval()
print(' '.join(source_sentence))
print()
x = TEXT.numericalize([source_sentence]).to(device).squeeze(1)
generated_sequence =model.generate_sequence(x)
words = [TEXT.vocab.itos[word_idx] for word_idx in generated_sequence]
print(' '.join(words))

['i', 'really', 'need', 'a']
i really need a

i really need a book , do now . to an i might i sit down i might i sit down i might i
